# Introduction

This is a companion notebook that aims to generate gif images from the prediction results.

This gif images will facilitate our understanding of the model's outputs.

In [8]:
# importing editor from movie py
import pandas as pd
from moviepy.editor import ColorClip, VideoFileClip, CompositeVideoClip, TextClip
import glob
from utils import pretty_display

In [63]:
def combine_clips(main_clip, result_df):
    """
    Create a composite clip object that contains the prediction and true labels
    """
    true_txt_clips = []
    for timestamp in result_df["true_label"].tolist():
        if pd.notna(timestamp):
            true_txt_clip = TextClip("True serve", fontsize = 75, color = 'white')
            true_txt_clip = true_txt_clip.set_position(("left", "bottom")).set_start(timestamp).set_duration(3) 
            true_txt_clips.append(true_txt_clip)

    pred_txt_clips = []
    
    for timestamp, confidence in zip(result_df["pred"].tolist(),result_df["confidence"].tolist()) :
        if pd.notna(timestamp):
            pred_txt_clip = TextClip(f"Pred serve ({confidence:.3f})", fontsize = 75, color = 'yellow')
            pred_txt_clip = pred_txt_clip.set_position(("right", "bottom")).set_start(timestamp).set_duration(3) 
            pred_txt_clips.append(pred_txt_clip)
            
    composite_clip = CompositeVideoClip([main_clip] + true_txt_clips + pred_txt_clips)
    return composite_clip

In [9]:
# Load result_df, a dataframe that contains predictions and true labels
RESULT_CSV_FOLDER = "result_csv"
result_dict = {}
result_csvs = glob.glob(f"{RESULT_CSV_FOLDER}/*")
for result_csv in result_csvs:
    video_ID = result_csv.split("/")[-1][:-4]
    print(f"Analyzing {video_ID}")
    result_df = pd.read_csv(result_csv)
    result_dict[video_ID] = {}
    result_dict[video_ID]["result_df"] = result_df

Analyzing Aspire_18_Premier_vs_OK_Charge_18_UA_2022-02-18
Analyzing Rukkus_vs__TVT_2022-01-22
Analyzing MxtuVgTAgnhtsDaqVF1
Analyzing RVA_15_Elite_v_Saddleback_15
Analyzing Havoc_14_1_vs_RVA_14_Black_2022-01-09


In [64]:
# Load the videos into clip objects and label the clips with the true labels and the predictions
VIDEO_FOLDER = "trunc_video"
for video_ID in list(result_dict.keys()):
    video_path = VIDEO_FOLDER +  "/" + video_ID + ".mp4"
    print(video_path)
    main_clip = VideoFileClip(video_path)
    main_clip = main_clip.without_audio()
    
    result_df = result_dict[video_ID]["result_df"]
    composite_clip = combine_clips(main_clip, result_df)
    result_dict[video_ID]["clip"] = composite_clip

trunc_video/Aspire_18_Premier_vs_OK_Charge_18_UA_2022-02-18.mp4
trunc_video/Rukkus_vs__TVT_2022-01-22.mp4
trunc_video/MxtuVgTAgnhtsDaqVF1.mp4
trunc_video/RVA_15_Elite_v_Saddleback_15.mp4
trunc_video/Havoc_14_1_vs_RVA_14_Black_2022-01-09.mp4


In [65]:
# Create gif images for all instances of false negative/false positive
GIF_FOLDER = "gif"
for video_ID in list(result_dict.keys()):
    result_df = result_dict[video_ID]["result_df"]
    clip = result_dict[video_ID]["clip"]
    for idx, row in result_df.loc[result_df.label != "true positive"].iterrows():
        time = row["pred"]
        if np.isnan(time):
            time = row["true_label"]
        label = row["label"]
        print(video_ID)
        print(label)
        print(pred_time)
        clip.subclip(time -1, time + 4).write_gif(f"{GIF_FOLDER}/{idx}_{video_ID}_{label}.gif", fps=15)

t:  41%|███████████████████████▏                                | 31/75 [39:39<00:05,  8.54it/s, now=None]

Aspire_18_Premier_vs_OK_Charge_18_UA_2022-02-18
false positive
1387.0
MoviePy - Building file gif/15_Aspire_18_Premier_vs_OK_Charge_18_UA_2022-02-18_false positive.gif with imageio.



t:  41%|███████████████████████▏                                | 31/75 [39:49<00:05,  8.54it/s, now=None]

Rukkus_vs__TVT_2022-01-22
false negative
1387.0
MoviePy - Building file gif/1_Rukkus_vs__TVT_2022-01-22_false negative.gif with imageio.



t:  41%|███████████████████████▏                                | 31/75 [39:59<00:05,  8.54it/s, now=None]

...


t:  41%|███████████████████████▏                                | 31/75 [40:10<00:05,  8.54it/s, now=None]

MxtuVgTAgnhtsDaqVF1
false positive
1387.0
MoviePy - Building file gif/0_MxtuVgTAgnhtsDaqVF1_false positive.gif with imageio.



t:  41%|███████████████████████▏                                | 31/75 [40:19<00:05,  8.54it/s, now=None]

MxtuVgTAgnhtsDaqVF1
false positive
1387.0
MoviePy - Building file gif/1_MxtuVgTAgnhtsDaqVF1_false positive.gif with imageio.



t: 100%|█████████████████████████████| 75/75 [00:08<00:00,  8.35it/s, now=None]
                                                                               

In [ ]:

#result.set_duration(200).write_videofile("test.mp4" ,audio=False, fps=2)
#result.subclip(0, 3).write_gif("false_pos2.gif", fps=15)

# Notes
`'Aspire_18_Premier_vs_OK_Charge_18_UA_2022-02-18'` -> mid

`Rukkus_vs__TVT_2022-01-22` -> mid


`'MxtuVgTAgnhtsDaqVF1' -> mid
